# Aufgabe 3 - Deterministisch dynamisches Lagerhaltungsmodell

## Mathematisches Modell

**Zielfunktion**

\begin{equation}
	minimiere \ \ K = \sum^{T}_{t=1} (k_{B} \cdot y_{t}(r_{t}) + k_{L} \cdot L_{t+1})
\end{equation}

**unter den Nebenbedingungen**

\begin{align}
&&  L_{t+1} &= L_{t} +  r_{t} - B_{t}  && \forall t \\[5pt]
&&  0 &\leq r_{t} \leq L_{t+1} + B_{t} && \forall t \\[5pt]
&&   L_{1} &= L_{T+1} = 0 && \\[5pt]
&& 1000 \cdot y_t &\geq r_t && \forall t \\[10pt]
&& L_t &\geq 0 && \forall t \\[5pt]
&& y_{t} &\in \{0,1\} && \forall t
\end{align}

## Daten

### Bedarfswerte

| Periode              |  1  |  2 |  3 |  4 |  5  |  6  |  7  |  8  |  9  | 10 | 11 | 12 |
|----------------------|:---:|:--:|:--:|:--:|:---:|:---:|:---:|:---:|:---:|:--:|:--:|:--:|
| **Mittelwerte**      |  92 | 40 | 71 | 83 | 130 | 148 | 135 |  95 | 122 | 65 | 50 | 45 |
| **Minimaler Bedarf** |  75 | 37 | 59 | 60 | 101 | 135 | 100 |  88 | 116 | 62 | 40 | 44 |
| **Maximaler Bedarf** | 112 | 45 | 81 | 95 | 132 | 163 | 140 | 103 | 145 | 70 | 68 | 53 |

### Weitere Daten

Ein Industrieunternehmen benötigt eine spezielle Materialart in der Produktion. Bestellungen können immer zu Beginn eines Monats ausgeführt werden und die Lieferung erfolgt sofort. Dabei verursacht jede Bestellung Kosten in Höhe von 400 GE. Im Gegenzug fallen für die Lagerung einer Einheit Aufwendungen in Höhe von 2 GE pro Periode an.

## Aufgabe 3b)
Übertragen Sie die gegebenen Daten. Verwenden Sie dazu die Datei `Bedarfe.csv`.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import csv

### Importieren der erforderlichen Daten

In [ ]:
B = {}
B["min"] = []
B["mean"] = []
B["max"] = []
with open("Bedarfe.csv", encoding="utf-8") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        B["min"].append(int(row["min"]))
        B["mean"].append(int(row["mean"]))
        B["max"].append(int(row["max"]))

In [ ]:
kL = 2
kB = 400

### Definition der Mengen

In [ ]:
T_max = len(B["mean"])
T = range(T_max)

## Aufgabe 3c)
Übertragen Sie das gegebene Modell und führen Sie es mit den Mittelwerten aus. Verwenden Sie dabei die mengenbasierte Syntax. Benennen Sie die Nebenbedingungen sinnvoll.

### Initialisierung des Modells

In [ ]:
m = gp.Model()

### Initialisierung der Variablen

In [ ]:
r = {}
for t in T:
    r[t] = m.addVar(vtype=GRB.INTEGER, lb=0, name="r" + str(t))

In [ ]:
y = {}
for t in T:
    y[t] = m.addVar(vtype=GRB.BINARY, name="y" + str(t))

In [ ]:
L = {}
for t in range(T_max + 1): # Da nach der letzten Periode der Lagerbestand wieder 0 sein soll, muss eine zusätzliche Periode hinzugefügt werden.
    L[t] = m.addVar(vtype=GRB.INTEGER, lb=0, name="L" + str(t))

### Definition der Zielfunktion

In [ ]:
m.setObjective(gp.quicksum(kB * y[t] + kL*L[t+1] for t in T), GRB.MINIMIZE)

### Hinzufügen der Nebenbedingungen

In [ ]:
for t in T:
    m.addConstr(L[t+1] == L[t] + r[t] - B["mean"][t], "LB" + str(t))

In [ ]:
for t in T:
    m.addConstr(r[t] <= L[t+1] + B["mean"][t], "BM" + str(t))

In [ ]:
for t in T:
    m.addConstr(y[t]*1000 >= r[t], "BK" + str(t))

In [ ]:
m.addConstr(L[0] == 0, "LAB")
m.addConstr(L[T_max] == 0, "LEB")

### Optimierung

In [ ]:
m.optimize()

### Ergebnisausgabe

In [ ]:
m.printAttr("ObjVal")

In [ ]:
m.printAttr("X")

## Aufgabe 3d)
Lassen Sie sich die .lp-Datei ausgeben und untersuchen Sie in welchen Nebenbedingungen und in welcher Form die Bedarfswerte verwendet werden.

In [ ]:
m.write("model.lp")
# Bedarfswerte sind mit ihrem negativem Wert die RHS der Lagerbilanzgleichung und mit ihrem poitiven Wert die RHS der Bestellmengenrestriktion.

## Aufgabe 3e)
Nutzen Sie Ihre Erkenntnisse aus der vorherigen Teilaufgabe und lassen Sie das Modell für alle Bedarfe (Minimal, Mittelwert, Maximal) lösen. Passen Sie dazu ggf. bei einigen Nebenbedingungen die RHS (rechte Hand Seite) an.

In [ ]:
for key in B.keys():
    for t in T:
        constrLB = m.getConstrByName("LB" + str(t))
        constrLB.setAttr(GRB.Attr.RHS, -B[key][t])
        constrBM = m.getConstrByName("BM" + str(t))
        constrBM.setAttr(GRB.Attr.RHS, B[key][t])
    m.optimize()

## Aufgabe 3f)
Speichern Sie zusätzlich alle berechneten Ergebnisse mit Kosten, Bestell- und Lagerpolitik gesammelt in geeignete Datentypen, sodass alle Ergebnis zusammengefasst ausgegeben werden können.

In [ ]:
costs = {}
yResults = {}
LResults = {}
rResults = {}

In [ ]:
for key in B.keys():
    for t in T:
        constrLB = m.getConstrByName("LB" + str(t))
        constrLB.setAttr(GRB.Attr.RHS, -B[key][t])
        constrBM = m.getConstrByName("BM" + str(t))
        constrBM.setAttr(GRB.Attr.RHS, B[key][t])
    m.optimize()
    costs[key] = m.ObjVal
    yResults[key] = []
    LResults[key] = []
    rResults[key] = []
    for t in T:
        yResults[key].append(round(y[t].X)) # Sonst wird teilweise -0 ausgegeben.
        LResults[key].append(L[t].X)
        rResults[key].append(r[t].X)

In [ ]:
print("\nAlle Bedarfe\n")
for key in costs.keys():
    print("Bedarfe:", key)
    print("Kosten:", costs[key])
    print("Bestellungen:", yResults[key])
    print("Lagerbestand:", LResults[key])
    print("Bestellmenge:", rResults[key],"\n")